# IFSP - Câmpus Campinas

## Análise sobre a previsibilidade das decisões de admissibilidade no Superior Tribunal de Justiça usando inteligência artificial

#### Aluno: Guilherme Cioldin Dainese

### Parte 2 - Pré-processamento e filtragem

Seguindo do arquivo gerado na parte anterior, essa parte consiste no pré-processamento, limpeza e análise exploratória do dataset obtido, bem como filtragem dos dados para seleção dos candidatos para preparação do corpus de treinamento.

### 1. Pré-processamento

In [1]:
#Carregando módulos utilizados nesse notebook
from pytz import timezone
import pandas as pd
pd.set_option('future.no_silent_downcasting', True) #Opção para o Pandas não dar FutureWarning (o código está adequado ao novo modo já)

In [2]:
df = pd.read_feather('./dataframes/json_raw.feather')
print(f'O dataset possui {df.shape[0]} entradas em {df.shape[1]} colunas/features.')

O dataset possui 751914 entradas em 12 colunas/features.


In [3]:
df.head()

,seqDocumento,dataPublicacao,tipoDocumento,numeroRegistro,processo,dataRecebimento,dataDistribuicao,ministro,recurso,teor,descricaoMonocratica,assuntos
0,163410022,1662001200000,ACÓRDÃO,202103427345,REsp 2005877,1634871600000,1635303600000,NANCY ANDRIGHI,None,Concedendo,None,5825;5825
1,162915181,1662001200000,ACÓRDÃO,202102019306,REsp 1946637,1624849200000,1626318000000,MANOEL ERHARDT (DESEMBARGADOR CONVOCADO DO TRF5),AGRAVO INTERNO,Concedendo,None,6060;6060;5994
2,163409979,1662001200000,ACÓRDÃO,202102897496,REsp 1959435,1630638000000,1632366000000,NANCY ANDRIGHI,None,Concedendo,None,10470;10470
3,163239479,1662001200000,ACÓRDÃO,201301198451,REsp 1382933,1366945200000,1371006000000,ASSUSETE MAGALHÃES,AGRAVO INTERNO,Negando,None,10305;10221
4,163227292,1662001200000,ACÓRDÃO,202201220906,REsp 1999207,1651201200000,1653015600000,MARCO BUZZI,AGRAVO INTERNO,Negando,None,10496;10496;14919;14920


#### 1.1 Removendo entradas duplicadas

O campo `seqDocumento` é o identificador principal do conjunto, vez que se refere a uma decisão específica. Usaremos ele como campo principal para remover eventuais entradas duplicadas.

In [4]:
#Remove entradas duplicadas, utilizando o 'seqDocumento' como chave
df.drop_duplicates(subset=['seqDocumento'], keep = 'last', inplace=True, ignore_index=True)
print(f'O dataset agora possui {df.shape[0]} entradas em {df.shape[1]} colunas/features.')

O dataset agora possui 739612 entradas em 12 colunas/features.


In [5]:
#Exibe os tipos de dados do dataset
df.dtypes

seqDocumento             int64
dataPublicacao           int64
tipoDocumento           object
numeroRegistro           int64
processo                object
dataRecebimento          int64
dataDistribuicao         int64
ministro                object
recurso                 object
teor                    object
descricaoMonocratica    object
assuntos                object
dtype: object

#### 1.2 Convertendo os horários

As colunas `'dataPublicacao'`, `'dataRecebimento'` e `'dataDistribuicao'` são de data, porém estão como int64. Para isso iremos ajustar os tipos de dados no pandas para datetime.

De se observar que simplesmente usar o módulo 'to_datetime' deixa os horários errados pois têm que ser considerado o horário de Brasília mas ele simplemsmente converte para hora UTC.
Por isso usaremos o tz_convert para converter no horário UTC-3 do Brasil.

In [6]:
brtz = timezone('America/Sao_Paulo') #Ajusta a timezone para correção da hora no dataset
df['dataPublicacao'] = pd.to_datetime(df['dataPublicacao'], unit='ms',utc=True)
df['dataRecebimento'] = pd.to_datetime(df['dataRecebimento'], unit='ms',utc=True)
df['dataDistribuicao'] = pd.to_datetime(df['dataDistribuicao'], unit='ms',utc=True)
df['dataPublicacao'] = df['dataPublicacao'].dt.tz_convert(brtz)
df['dataRecebimento'] = df['dataRecebimento'].dt.tz_convert(brtz)
df['dataDistribuicao'] = df['dataDistribuicao'].dt.tz_convert(brtz)
df.tail()

,seqDocumento,dataPublicacao,tipoDocumento,numeroRegistro,processo,dataRecebimento,dataDistribuicao,ministro,recurso,teor,descricaoMonocratica,assuntos
739607,224027345,2024-01-14 23:00:00-03:00,DECISÃO,202301309450,AREsp 2343988,2023-04-20 00:00:00-03:00,2023-05-19 00:00:00-03:00,REYNALDO SOARES DA FONSECA,None,Concedendo,Agravo conhecido para dar parcial provimento a...,10621;3637;5555;5566;10621;12334
739608,223999858,2024-01-14 23:00:00-03:00,DECISÃO,202301132414,HC 814406,2023-04-04 00:00:00-03:00,2023-04-10 00:00:00-03:00,REYNALDO SOARES DA FONSECA,None,Concedendo,Não conhecido o Habeas Corpus. Concedido o Hab...,3628;3628;3628
739609,224007770,2024-01-14 23:00:00-03:00,DECISÃO,202301118658,AREsp 2330257,2023-04-04 00:00:00-03:00,2023-04-13 00:00:00-03:00,REYNALDO SOARES DA FONSECA,AGRAVO REGIMENTAL,Concedendo,Agravo conhecido para dar provimento ao Recurs...,3372;3372;10637
739610,224040816,2024-01-14 23:00:00-03:00,DECISÃO,202002489460,AREsp 1764530,2020-09-22 00:00:00-03:00,2020-10-06 00:00:00-03:00,SEBASTIÃO REIS JÚNIOR,None,Concedendo,Agravo conhecido para dar provimento ao Recurs...,3608;3608
739611,224056232,2024-01-14 23:00:00-03:00,DECISÃO,202400008931,SLS 3387,2024-01-08 23:00:00-03:00,2024-01-08 23:00:00-03:00,MARIA THEREZA DE ASSIS MOURA,None,Concedendo,Julgado procedente o pedido,10073;10073


As decisões constam com data de publicação, mas como se pode ver no excerto do dataframe, as datas de recebimento são muito variadas.

Isso se dá porque os processos não são necessariamente julgados em ordem cronológica.

Em razão disso eliminaremos as entradas com data de *recebimento* anteriores à 01/09/2022.

A data de Distribuição não é utilizada pois a distribuição ocorre posterior ao recebimento.

In [7]:
# Filtra para excluir entradas anteriores à data de corte
df = df[df['dataRecebimento'] >= pd.Timestamp('2022-09-01 00:00:00', tz='America/Sao_Paulo')]
df = df.reset_index(drop=True)
print(f'O dataset agora possui {df.shape[0]} entradas em {df.shape[1]} colunas/features.')

O dataset agora possui 496150 entradas em 12 colunas/features.


#### 1.3 Organizando a coluna `processo`

A coluna `processo` possui dois tipos de dado em uma única coluna; a classe processual e o número de ordem dessa classe processual dentro da Corte.

Para tanto é necessário separar os tipos de dados presente nesse campo, inclusive para obtermos a classe processual.

In [8]:
#Criando a nova coluna para o número
df.insert(df.columns.get_loc('processo') + 1, 'numeroProcesso', None)

In [9]:
#Extraindo o número para a coluna numeroProcesso
df['numeroProcesso'] = df['processo'].str.extract(r'(\s\d+)')
df['processo'] = df['processo'].str.replace(r'(\s\d+)', '',regex=True)
df['numeroProcesso'] = df['numeroProcesso'].astype('int64')

In [10]:
#Removendo os espaços em branco remanescentes
df['processo'] = df['processo'].str.replace(r'\s+', '', regex=True)

In [11]:
#Renomeando a coluna
df = df.rename(columns={'processo': 'classeProcesso'})

In [12]:
df.dtypes

seqDocumento                                        int64
dataPublicacao          datetime64[ns, America/Sao_Paulo]
tipoDocumento                                      object
numeroRegistro                                      int64
classeProcesso                                     object
numeroProcesso                                      int64
dataRecebimento         datetime64[ns, America/Sao_Paulo]
dataDistribuicao        datetime64[ns, America/Sao_Paulo]
ministro                                           object
recurso                                            object
teor                                               object
descricaoMonocratica                               object
assuntos                                           object
dtype: object

#### 1.4 Fazendo o *encoding* da coluna `tipoDocumento`

A coluna tipoDocumento possui apenas dois valores (DECISÃO e ACÓRDÃO).

Ela pode ser representada numericamente. Convém fazer o *encoding* dos valores.

In [13]:
#Confirmando os tipos de dados
df['tipoDocumento'].value_counts()

tipoDocumento
DECISÃO    419066
ACÓRDÃO     77084
Name: count, dtype: int64

In [14]:
#Substituindo os valores e renomeando a coluna
df['tipoDocumento'] = df['tipoDocumento'].replace({'ACÓRDÃO': 1, 'DECISÃO': 0}).infer_objects(copy=False)
df = df.rename(columns={'tipoDocumento': 'isAcordao'})

In [15]:
df['isAcordao'].value_counts()

isAcordao
0    419066
1     77084
Name: count, dtype: int64

#### 1.5 Organizando a coluna `assuntos`

A coluna assuntos é especialmente relevante para podermos selecionar assuntos similares e fazermos comparações justas entre as decisões concedendo e negando. 

Porém um simples exame do dataset demonstra que alguns valores estão repetidos. Para tanto é necessário separá-los e organizá-los.

In [16]:
#Criando uma função simples para organizar os repetidos
def assuntos_unicos(entrada):
    assuntos = entrada.split(',')
    return ','.join(sorted(list(set(assuntos))))

In [17]:
#Aplicando a função no dataset
df['assuntos'] = df['assuntos'].str.replace(' ', '')
df['assuntos'] = df['assuntos'].str.replace(';', ',')
df['assuntos'] = df['assuntos'].apply(lambda x: assuntos_unicos(x))

In [18]:
df['assuntos'].value_counts()

assuntos
3608                   35085
5566                   10271
3372                    8217
3608,5897               7253
9607                    6018
                       ...  
5994,6011,6056,6060        1
10158,6017,6085            1
3572,3608,5885             1
10425,10582,9596           1
10671,11802,11839          1
Name: count, Length: 55918, dtype: int64

#### 1.6 Limpando a coluna descrição monocrática

Algumas entradas no campo descrição monocrática possuem texto a ser limpo.

In [19]:
df['descricaoMonocratica'] = df['descricaoMonocratica'].str.replace('\r\n', '', regex=False)

### 2. Filtragem dos dados

O *dataset* está relativamente organizado para seguirmos com o projeto.

Agora precisamos filtrar os que seguirão para análise.

Importante lembrar que a filtragem pela data foi realizada na etapa 1.2, acima.

#### 2.1 Filtragem pela classe processual

Primeiramente, é necessário selecionar os tipos de processos relevantes.

Selecionaremos somente os processos de Recurso Especial e Agravos em Recurso Especial pois são os únicos que tratam diretamente do recurso objeto de estudo.

In [20]:
df = df.loc[df['classeProcesso'].isin(['AREsp', 'REsp'])].reset_index(drop=True)
df['classeProcesso'].value_counts()

classeProcesso
AREsp    284447
REsp      75549
Name: count, dtype: int64

#### 2.2 Filtragem pelos processos únicos

Também convém eliminar os processos que possuem incidentes, indicados na coluna `recurso`.

In [21]:
print(f'Número de processos antes da limpeza: {df.shape[0]}\n')
df.drop_duplicates(subset='numeroRegistro', keep=False, inplace=True, ignore_index=True) #keep=False elimina as duas entradas de dados duplicados.
print(f'Número de após a limpeza: {df.shape[0]}')

Número de processos antes da limpeza: 359996

Número de após a limpeza: 204721


#### 2.3 Filtragem dos processos com incidentes

A etapa anterior removeu processos que continham incidentes processuais. Porém, restam processos que somente constam os incidentes no dataset.

Isso se dá porque inicalmente somente selecionamos or processos após setembro de 2022 para compor o *dataset*.

Os que foram recebidos antes da data de corte mas julgados após setembro não foram excluídos na etapa anterior. Precisamos excluí-los também.

In [22]:
df = df[df['recurso'].isnull()].reset_index(drop=True)
df.drop(columns=['recurso'], inplace=True)
df.shape

(203265, 12)

#### 2.4 Filtrando por assuntos

Nesse momento precisamos filtrar os assuntos que comporão o dataset final.

O segundo assunto mais presente nas decisões é o assunto '9607'. Consultando no sistema do CNJ (https://www.cnj.jus.br/sgt/consulta_publica_assuntos.php), verifica-se que esse assunto é um tema específico na tabela (não possui subtemas).

Isso é importante pois diminuímos os efeitos que a discussão do mérito processual tem na análise das petições e, por ser um tema de direito civil, garantimos que todos os processos seguem o mesmo rito, o que não aconteceria se comparassemos a processos criminais, de improbidade administrativa ou afins, que possuem ritos diversos.

In [23]:
df['assuntos'].value_counts()

assuntos
3608                      6354
9607                      3805
5566                      2782
6107                      2732
7752                      2601
                          ... 
10236,8829                   1
10502,14033,9992             1
10074,7691,9418              1
10556,5938                   1
10687,10873,10874,5994       1
Name: count, Length: 34120, dtype: int64

In [24]:
#Criando função para selecionar o assunto pretendido
#O ValueError ocorre se o formato não for int, o que significa que é uma lista com vários assuntos.
#Logo, é uma lista e verificamos se o assunto buscado consta na lista.
def busca_assunto(valor):
    assunto = 9607
    try:
        valor = int(valor)
        return valor == assunto
    except ValueError: 
        valor = list(map(int, valor.split(',')))
        return assunto in valor

In [25]:
df = df[df['assuntos'].apply(busca_assunto)]
print(f'Número de processos com o assunto selecionado: {df.shape[0]}')

Número de processos com o assunto selecionado: 6411


#### 2.5 Preparando a coluna `y`

Aqui preparamos a coluna com o valor alvo para o treinamento dos modelos.

O objetivo é classificar as petições que expuseram argumentos que levaram a aceitação do recurso especial.

Para tanto podemos classificar da seguinte maneira:
```
Para AREsps:
    Se o AREsp for não conhecido, o valor é 0.
    Se o AREsp for conhecido e não provido, o valor é 0.
    Se o AREsp for conhecido e provido (ainda que provido só parcialmente), o valor é 1.

Para REsps:
    Se o REsp não foi conhecido.
    Se o REsp for conhecido (mesmo que conhecido parcialmente, independentemente se foi provido ou não provido), o valor é 1.
```

In [26]:
df['teor'].value_counts()

teor
Não Conhecendo    3932
Negando           1756
Concedendo         601
Outros              88
Deferindo            2
Name: count, dtype: int64

In [27]:
#Removendo outras decisões
df = df[df['teor'] != 'Outros']
df = df.dropna(subset=['teor'])

In [28]:
#Examinando as duas decisões que consta como 'Deferindo'
df[df['teor'] == 'Deferindo']

,seqDocumento,dataPublicacao,isAcordao,numeroRegistro,classeProcesso,numeroProcesso,dataRecebimento,dataDistribuicao,ministro,teor,descricaoMonocratica,assuntos
51997,181525945,2023-03-16 00:00:00-03:00,1,202202963138,REsp,2027862,2022-09-16 00:00:00-03:00,2022-09-30 00:00:00-03:00,NANCY ANDRIGHI,Deferindo,None,9607
121750,202734965,2023-08-14 00:00:00-03:00,1,202301022078,REsp,2062295,2023-03-28 00:00:00-03:00,2023-04-19 00:00:00-03:00,NANCY ANDRIGHI,Deferindo,None,"10735,10737,7691,8942,9607"


Se tratam de dois recursos especiais. Se foram deferidos, foram admitidos, logo o valor de y é 1 para estes.

Para fins de organização, vamos mudar o valor dessa coluna para Concedendo.

In [29]:
df['teor'] = df['teor'].replace('Deferindo', 'Concedendo')
df['teor'].value_counts()

teor
Não Conhecendo    3932
Negando           1756
Concedendo         603
Name: count, dtype: int64

In [30]:
#Criando a coluna y
df['y'] = 0

In [31]:
#Verificando os valores para processos da classe REsp
df[df['classeProcesso'] == 'REsp']['teor'].value_counts()

teor
Concedendo        376
Não Conhecendo    199
Negando           171
Name: count, dtype: int64

In [32]:
#Coloca o y de acordo com os critérios acima para a coluna REsp.
df['y'] = df.apply(lambda row:
                   1 if row['classeProcesso'] == 'REsp' 
                   and row['teor'] == 'Concedendo' 
                   else row['y'], axis=1)
df['y'] = df.apply(lambda row:
                   1 if row['classeProcesso'] == 'REsp' 
                   and row['teor'] == 'Negando' 
                   else row['y'], axis=1)
df[df['classeProcesso'] == 'REsp']['y'].value_counts()

y
1    547
0    199
Name: count, dtype: int64

A substituição foi realizada corretamente.

Resta examinar as decisões do Agravo em Recurso Especial.

In [33]:
#Verificando os valores para processos da classe AREsp
df[df['classeProcesso'] == 'AREsp']['teor'].value_counts()

teor
Não Conhecendo    3733
Negando           1585
Concedendo         227
Name: count, dtype: int64

In [34]:
#Coloca o y de acordo com os critérios acima para a coluna AREsp.
df['y'] = df.apply(lambda row:
                   1 if row['classeProcesso'] == 'AREsp' 
                   and row['teor'] == 'Concedendo' 
                   else row['y'], axis=1)
df[df['classeProcesso'] == 'AREsp']['y'].value_counts()

y
0    5318
1     227
Name: count, dtype: int64

In [35]:
df['y'].value_counts()

y
0    5517
1     774
Name: count, dtype: int64

#### 2.7 Salvando o dataframe para as etapas seguintes

O pré-processamento e filtragem estão completos, o dataframe pode ser salvo para as etapas seguintes.

In [36]:
df.to_feather('./dataframes/filtered_dataframe.feather')